In [134]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import RNN
from keras.utils import np_utils
from os import path

In [135]:
import re
text = (open("amloClean.txt", encoding="utf-8").read())
text = text.lower()
text = re.sub(r"http\S+", "", text)
#text = bytes(text, "utf-8").decode("unicode_escape")
text

'después de 20 días de campaña tengo tiempo para estar en la casa, y pegar estampitas que le traje a jesús de la gira por el norte.\nya no soy peje, ahora soy andresmanuelovich\nmeade, además de tapadera, es un calumniador. anexo las pruebas para demostrar que los "tres departamentos" no son míos. él ya lo sabía, pero anda muy nervioso, lo comprendo.\naquí pobremente echando novio.\nvamos a ganar el día primero de julio y no vamos a fallarle al pueblo. el poder solo tiene sentido, y se convierte en virtud, cuando se pone al servicio de los demás.\n¡gracias!\nagradezco al doctor rodolfo neri vela su confianza. es un gran científico, el único astronauta mexicano y está demostrando ser un buen ciudadano. no lo decepcionaré y siempre estaré a la altura de quienes, como él, desean un cambio verdadero.\nestuvimos en milpa alta y xochimilco, miren la plaza de este último lugar, histórico y chinampero.\nmiren cómo nos fue en nogales. como diría juan gabriel ‘a mí me gusta mucho estar en la fro

In [136]:
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

In [137]:
X = []
Y = []
length = len(text)
seq_length = 100

In [138]:
for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [139]:
model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

'''
model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
'''

"\nmodel = Sequential()\nmodel.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))\nmodel.add(Dropout(0.2))\nmodel.add(LSTM(400))\nmodel.add(Dropout(0.2))\nmodel.add(Dense(Y_modified.shape[1], activation='softmax'))\nmodel.compile(loss='categorical_crossentropy', optimizer='adam')\n"

In [140]:
model.fit(X_modified, Y_modified, epochs=100, batch_size=50)
model.save_weights("amlo_text_generator_400_0.2_400_0.2_deeper.h5")

Epoch 1/100
16260/16260 [==============================] - 678s 42ms/step - loss: 3.0534
Epoch 2/100
16260/16260 [==============================] - 673s 41ms/step - loss: 3.0074
Epoch 3/100
16260/16260 [==============================] - 675s 42ms/step - loss: 2.8769
Epoch 4/100
16260/16260 [==============================] - 670s 41ms/step - loss: 2.7809
Epoch 5/100
16260/16260 [==============================] - 672s 41ms/step - loss: 2.7314
Epoch 6/100
16260/16260 [==============================] - 670s 41ms/step - loss: 2.6756
Epoch 7/100
16260/16260 [==============================] - 671s 41ms/step - loss: 2.6210
Epoch 8/100
16260/16260 [==============================] - 667s 41ms/step - loss: 2.0470
Epoch 15/100
16260/16260 [==============================] - 673s 41ms/step - loss: 1.8938
Epoch 16/100
16260/16260 [==============================] - 672s 41ms/step - loss: 1.7306
Epoch 17/100
16260/16260 [==============================] - 673s 41ms/step - loss: 1.5595
Epoch 18/100
16260

16260/16260 [==============================] - 658s 40ms/step - loss: 0.0738
Epoch 100/100
16260/16260 [==============================] - 661s 41ms/step - loss: 0.0935


In [141]:
#model.load_weights('amlo_text_generator_400_0.2_400_0.2_baseline.h5')
model.load_weights('amlo_text_generator_400_0.2_400_0.2_deeper.h5')

In [150]:
new_text = "la mafia del poder quiere calumniar y acusarme de cosas que no he hecho. es un hecho irrefutable que q"

new_X = []
new_length = len(new_text)
seq_length = len(new_text)-2

for i in range(0, new_length-seq_length, 1):
    sequence = new_text[i:i + seq_length]
    label = new_text[i + seq_length]
    new_X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

string_mapped = new_X[1]
full_string = [n_to_char[value] for value in string_mapped]

#print(full_string)
    
for i in range(1000):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]
    
#print(string_mapped)

In [151]:
#combining text
txt=""
for char in full_string:
    txt = txt+char
txt

'a mafia del poder quiere calumniar y acusarme de cosas que no he hecho. es un hecho irrefutable que seli ma iicie podre reges está la gente "ahora sí" o como lo dice la canción de amaury pérez \'no lo van a impedir\'.\nel sábado propondré en consejo de morena que se destine 50% del gasto de campaña de 2018 para los damnificados del istmo y del sureste del país.\nbuscaremos una relación de amistad y de cooperación con el gobierno de estados unidos, con respeto a nuestro pueblo y a la soberanía. los problemas sociales no se resuelven con muros ni con la militarización de la frontera, ni al muro; sí al respeto mutuo y a la cooperación para el desarrollo\nterminamos la gira por la frontera, además de reafirmar nuestra postura con relación a estados unidos, dimos a conocer nuestros proyectos para convertir a méxico en una potencia económica y social para el medio rural que se aplicará al triunfo de nuestro movimiento\nno seré mediocre ni mucho menos traidor a la patria. aspiro a seguir el 

# Texto generado de manera aleatoria

In [154]:
string_mapped = X[80]
full_string = [n_to_char[value] for value in string_mapped]
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

In [155]:
#combining text
txt=""
for char in full_string:
    txt = txt+char
txt

'itas que le traje a jesús de la gira por el norte.\nya no soy peje, ahora soy andresmanuelovich\nmeade, además de tapadera, es un calumniador. anexo las pruebas para demostrar que los "tres departamentos" no son míos. éa no say a caeo a fasor de 1 de junio y la gente va celebrar bailando "despajito"\nestamos a 9 meses del 1 de julio del 18. aunque no venga en la boleta, la pregunta implícita será ¿continuidad o cambio? hagamos  historia.\nesperemos que se cumpla el compromiso de transparentar los co'